# PyData Cardiff Workshop 3 - Introduction to Pandas

![title](images/pydata_cardiff.jpg)

## Introduction to the library

Pandas is a seminal python library, which has revolutionised data analytics for the programming language. It began development in 2008 by Wes McKinney when he was working at AQR Capital Management. Initially, it was a purely in-house project, but on leaving his position, Wes was able to convince AQR to permit him to open-source the code.

If anyone is interested - the name Pandas stands for PANel Data ANalysis

Note that the usual way to import this library is to use the pattern `import pandas as pd`

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Introducing the basic data types

### The Pandas Series

A one dimensional array of information. It has similarities with a numpy array - and it can be useful to think of a series like a column of information in an Excel Spreadsheet. Similarly to the numpy array - all of the data types in a series _should_ be of the same data type.

#### Creating a simple Series - very similar to a `numpy array`

Note the present of a single integer at the end - but this will be coerced to a float.

In [ ]:
ar = np.array([0.2, 1.2, 3.4, 5.6, 3.8, 6.7, 1.2, 7])
ser = pd.Series([0.2, 1.2, 3.4, 5.6, 3.8, 6.7, 1.2, 7])

In [ ]:
ar.dtype

In [ ]:
ser.dtype

## Note how the series deals with Mixed types

It states that they are of type `'O'` - meaning a Python object!

In [ ]:
object_ser = pd.Series([1, 'hello', None, 3.4])

In [ ]:
object_ser

In [ ]:
object_ser.dtype

### Similar methods and functionality

There are a series of methods of the Series that share the same functionality with numpy arrays. There are called the numpy 'universal' functions `ufunc`

In [ ]:
ar.mean()

In [ ]:
ser.mean()

In [ ]:
ar.sum()

In [ ]:
ser.sum()

### However!

There will be some different behaviours seen! Note the different ways in which the variance is calculated.

In numpy - this is calculated as:

$$\frac{\Sigma (x - \bar{x})^{2}}{n}$$

In [ ]:
ar.var()

But in the Series - this is calculated as the _unbiased_ variance, using a method called _Bessel's Correction_ by subtracting 1 from _n_

$$\frac{\Sigma (x - \bar{x})^{2}}{n - 1}$$

The effect that this has is a larger value for variance. In statistics - this has useful implications by making the variance of distributions wider, and statistical testing more rigorous.

This value can be set by chaning the _delta degrees of freedom_ argument `ddof`

In [ ]:
ser.var()

In [ ]:
ser.var(ddof=1)

In [ ]:
ser.var(ddof=0)

We can always call the series with `.values` to get the information as a numpy array

In [ ]:
isinstance(ser.values, np.ndarray)

## The Series Index

This is a key feature of the Series when compared with the array - and can be thought of as the name that the a row would be given if the Series was a column in a SpreadSheet.

This can be seen when we simply view the object - note that as we did not set this, the default value is the number of the row - indexed from 0

In [ ]:
ar

In [ ]:
ser

In [ ]:
ser.index

This can be set at the creation of the variable - and note that we can use the values of the previous series

In [ ]:
ser2 = pd.Series(data=ser.values, index = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'])

In [ ]:
ser2

In this way - the series can be interacted with in a similar fashion to a dictionary

In [ ]:
di = {'a': 0.2, 'g': 1.2, 'h': 7., 'b': 1.2, 'c': 3.4, 'f': 6.7, 'd': 5.6, 'e': 3.8}

In [ ]:
ser[2]

In [ ]:
ser2['c']

In [ ]:
di['c']

However - note that there is an additional slicing ability that is not present in dictionaries

__BUT__ - take care to notice that this slicing in Pandas is __inclusive__ of the end point!!!!

In [ ]:
ser2['c': 'f']

In [ ]:
di['c': 'f']

## Additional functionality in the Series

A good example of this are the functions `rolling` and `expanding`. These create a type of _Window_ function - either sliding or expanding.

Note the presence of the missing values when calling these functions. In this case, the first two values are first used to calculate the third value.

In [ ]:
ser2.expanding(3).mean()

In [ ]:
ser2.rolling(3).mean()

There is also functionality to shift the data by position

In [ ]:
ser2.shift(1)

In [ ]:
ser2.shift(-3)

## Missing values - differences between numpy and pandas

One feature of numpy arrays is that the presence of missing values can have a detrimental effect when performing any `func`

Note that we __must__ use the `np.nan` (not a number) variable to create the missing value - `None` will not work

In [ ]:
ar_missing = np.array([1, 2, 3, 4, np.nan, 5])

In [ ]:
ar_missing

In [ ]:
ar_missing.sum()

In [ ]:
ar_missing.mean()

Slight difference here:

In [ ]:
ar_missing.cumsum()

This has to be dealt with using the specialised functions

In [ ]:
np.nansum(ar_missing)

In [ ]:
np.nanmean(ar_missing)

In [ ]:
np.nancumsum(ar_missing)

In Pandas - these function __are the default!__

Also - note that we can create a missing value using `None` - it will get changed to a `NaN` automatically

In [ ]:
ser_missing = pd.Series([1, 2, 3, 4, None, 5])

In [ ]:
ser_missing

In [ ]:
ser_missing.sum()

In [ ]:
ser_missing.mean()

This one is slightly different!

In [ ]:
ser_missing.cumsum()

## Dealing with missing values

Missing values are a common feature of using real datasets. 4 examples of how to deal with these are shown here.

1. Replacing the missing value with a stated replacement
2. Backfilling the data from later/lower
3. Forward filling the data from earlier/higher
4. Just drop them entirely!

In [ ]:
ser_missing_start = ser2.shift(3)

In [ ]:
ser_missing_start

In [ ]:
ser_missing_start.fillna(-999)

Remember that missing numbers won't affect the mean calculation in Pandas

In [ ]:
ser_missing_start.fillna(ser_missing_start.mean())

In [ ]:
ser_missing_start.bfill()

In [ ]:
ser_missing_end = ser2.shift(-3)

In [ ]:
ser_missing_end

In [ ]:
ser_missing_end.ffill()

In [ ]:
ser_missing_end.dropna()

# Moving to the DataFrame

This is really the main datatype in Pandas. Think of one as a collection of Series objects - all sharing the same index.

A dataframe can be created using a variety of methods - only a few of which will be shown here.

Using a dictionary. However in order to maintain the desired column order - we will be using an `OrderedDict` here

In [ ]:
from collections import OrderedDict

In [ ]:
data1 = OrderedDict({
    'col1': [1, 2, 3, 4],
    'col2': [4, 5, 6, 7]
})

In [ ]:
df1 = pd.DataFrame(data1)

In [ ]:
df1

Using a numpy array, with column information

In [ ]:
data2 = np.array([
    [1, 4],
    [2, 5],
    [3, 6],
    [4, 7]
])

In [ ]:
data2

In [ ]:
df2 = pd.DataFrame(data2, columns=['col1', 'col2'])

In [ ]:
df2

In a similar fashion to a series - we can use `.values` to get the data as a numpy array

In [ ]:
df2.values

The index can also be set at creation

In [ ]:
df3 = pd.DataFrame(data1, index=['a', 'b', 'c', 'd'])

In [ ]:
df3

Note that the columns and index __must__ be of the correct length, or you will get an error!

In [ ]:
# err = pd.DataFrame(data1, index=['a', 'b', 'c'])

## Adding and selecting data

If we wish to add a column of information to the dataframe, we can use dictionary-like `[]`, just as long as the length of the value being assigned is of the correct length.

In [ ]:
df3['col3'] = [4, 3, 2, 1]
df3['col4'] = [101, 102, 103, 104]
df3['col5'] = [-1, -2, -3, -4]

In [ ]:
df3

We can also use the `[]` notation to obtain a single series back from the dataframe, using the column name

In [ ]:
col2_series = df3['col2']

In [ ]:
col2_series

In [ ]:
type(col2_series)

## Using double brackets - `[[]]`

A very important feature to learn is that, while the `[]` notation returned a series, if we use double square brackets, then we do not get a series... but a __dataframe__

In [ ]:
col2_df = df3[['col2']]

In [ ]:
col2_df

In [ ]:
type(col2_df)

As dataframes do not need to be 1D - we can use this method to select multiple columns

In [ ]:
df3[['col1', 'col3']]

## Using conditional statements to select information

* This can include either single - or multiple statements
* But note the syntax for how multiple statements are used

In [ ]:
df3[df3['col1'] > 1]

In [ ]:
df3[(df3['col1'] > 1) & (df3['col2'] >= 6)]

## Using the `.iloc` and `.loc` notation

This is often the preferred method of selecting data. It can seem a little strange - but this will hopefully break it down

* We use `loc` for using identifiers present in the index
* We use `iloc` when getting the numbers of the rows - indexed from 0
    * Of course - if the index is the default of row numbers - then this will be the same!
* The earlier feature of `[]` for series and `[[]]` still holds!

Of note - you will sometime see the func `ix` used in some older text - this has now been deprecated

In [ ]:
df3.loc['a']

In [ ]:
df3.iloc[0]

In [ ]:
df3.loc[['a']]

In [ ]:
df3.iloc[[0]]

## Selecting column  as well!

Note that this will return the value that appears in a particular cell

In [ ]:
df3.loc['a', 'col3']

## Slicing

Using this method - we can use slicing for both rows and columns

In [ ]:
df3.loc['a': 'd', 'col2': 'col4']

# The `SettingWithCopy` warning!

This will soon become the bane of your life when working with Pandas dataframes!

Here I will try to explain it as best as I can!

In [ ]:
df4 = df3.copy()

In [ ]:
df4

In [ ]:
df4['a': 'c']['col2']

In [ ]:
df4['a': 'c']['col2'] = [32, 31, 30]

It has still worked though!

In [ ]:
df4

In [ ]:
df4 = df3.copy()

Using `loc`, we avoid this error!

In [ ]:
df4.loc['a': 'c', 'col2'] = [32, 31, 30]

In [ ]:
df4

### Now - this seems to work here

In [ ]:
df5 = df4.loc['a': 'c', ['col1', 'col3', 'col5']]

In [ ]:
df5

In [ ]:
df5.loc['a', 'col3'] = 9999

In [ ]:
df6 = df4.loc['b': 'd', :]

In [ ]:
df6.loc['a', 'col3'] = 9999

### Just observe how irritating this is!

This really looks the same to me!

In [ ]:
warning_data = {'one': np.arange(1, 11), 'two': np.arange(11, 21)}   

In [ ]:
warning_df = pd.DataFrame(warning_data)

In [ ]:
warning_df

In [ ]:
warning_df2 = warning_df.loc[3:5, :] 

In [ ]:
warning_df2

In [ ]:
warning_df2.loc[4, 'one'] = 99 

In [ ]:
warning_df

### Just make a copy!

In [ ]:
no_warning_df = pd.DataFrame(warning_data)

In [ ]:
no_warning_df2 = no_warning_df.loc[3:5, :].copy()

In [ ]:
no_warning_df2.loc[4, 'one'] = 99 

In [ ]:
no_warning_df2

In [ ]:
no_warning_df

#### This is admittedly confusing! For a more detailed explanation - see [this blog](https://www.dataquest.io/blog/settingwithcopywarning/)

# Loading in data

This is probably the most important part of the workshop, as it will be one of the most common processes that you will __always__ do when carrying out data analysis. For this, we will look at loading in data from both a comma-separated-value file `.csv` and Excel files (other methods can include reading in streaming data - or information from relational databases). The format that you will probably be working with most is `.csv`. This is done using the following methods:

* `pd.read_csv()`
* `pd.read_excel()`
    * Note that to use this - you must install the `xlrd` library to read
    * And the `openpyxl` (together with its dependencies) to write data (use `pip` or `conda`)
        * But we won't be using that here!

This quickly can get more complicated that it initially sounds - a quick look at the documentation for these functions shows that! This is because of all of the potential problems that have to be considered when _parsing_ data from an external source. We do not have time to cover all of these, but a few of the features will be explained.

## Loading data from a _clean_ `.csv` file

* Note that this dataset does not have any index information - so one will be made with the row numbers indexed from 0
* Also - the file does not _have_ to separated by commas - if any other punctuation is use (like `;`), then this can be specified with the `delimiter` or `sep` argument (they do exactly the same thing - violation of the Zen of Python!)
    * If you know that your columns are segregated by spaces - or any other form of whitespace - then use the `delim_whitespace = True` in the function call

In [ ]:
!head -n 5 data/iris.csv

In [ ]:
iris_csv = pd.read_csv('data/iris.csv')

# This is just the same as:
# iris_csv = pd.read_csv('data', delimiter=',')

### Examining the data with `.head()` and `.tail()`

Probably the most used function that you will ever learn in Pandas is `head()`, which allows us to see the first 5 rows of data by default - but this number can be changed.

`tail()` has similar functionality - but shows the end of the dataframe rather than the top

In [ ]:
iris_csv.head()

In [ ]:
iris_csv.head(10)

In [ ]:
iris_csv.tail()

## Reading in from Excel

Here - the syntax is very similar, but note that as the file in question has multiple sheets - we can specify the sheet name of interest

In [ ]:
diamonds = pd.read_excel('data/iris_and_diamonds.xlsx', sheet_name='diamonds')

In [ ]:
diamonds.head()

## Loading in some _problematic_ data!

This file has 2 lines of junk information at the top of it - you will sometimes get it when downloading from certain sites - as they like to put it in there for identification purposes - and to make our work more interesting/unbelievably-irritating!

In [ ]:
!head -n 5 data/iris_problem.csv

In [ ]:
iris_csv2 = pd.read_csv('data/iris_problem.csv')

In [ ]:
iris_csv2 = pd.read_csv('data/iris_problem.csv', skiprows=2)

## Writing out the data

If we want to save our data into a text file - we can use the `to_csv()` function. The main thing to be careful of here is to make sure that you specify whether you want the index to be saved or not. by default - it will be, so if you don't have any meaningful information there, be sure to set it to False!

In [ ]:
# iris_csv2.to_csv('iris_with_index.csv')

In [ ]:
# iris_csv2.to_csv('iris_no_index.csv', index=False)

In reality though - we could probably just delete this in the text file before we load it in!

# Useful information functions

We will now look at 2 functions that can be used to get some summary information on the dataframes - we will stay with the iris dataset that we have loaded in already

## `info`

This will give us basic information about:

* The number of entries
* The number of missing values
* The data type of each column/series

In [ ]:
iris_csv.info()

## `describe`

This gives us a basic statistical summary of the data

* Note that by default - it will only include the numerical information
* If we want information on any `object` or 'categorical' columns - we need to include `include='all'` in the arguments

In [ ]:
iris_csv.describe()

In [ ]:
iris_csv.describe(include='all')

# Aggregating information

This is where the power of Pandas really comes into play. We can start performing database style operations on our data. Before Pandas - this was a particular headache!

Here, we will be using the values in the `species` column to get aggregated values of the information in the numerical columns. This is referred to as a __Group By__ operation.

Note that just calling a `groupby` returns a new type of object - this is because we have only _primed_ pandas to anticipate that we wish to do something with the grouping:

In [ ]:
iris_csv.groupby('species')

In this first example - we will take the average value of all of the numerical column by species. Note that the information that was in the species column __is now the index of the new dataframe!!!__

In [ ]:
iris_mean = iris_csv.groupby('species').mean()

In [ ]:
iris_mean

### In built functions

Notice that we only had to call the `mean` function on the grouped data, we can also use a variety of different methods

In [ ]:
iris_sum = iris_csv.groupby('species').sum()
iris_sum

In [ ]:
iris_std = iris_csv.groupby('species').std()
iris_std

## Using the generic `agg`

These are all suitable to do something that is built in, but if we wish to do something more complicated - then we have to use the generic `agg` method (aggregation)

In this first instance - we are performing multiple aggregations, and we now see the first example of a multi level column. We will be looking at how to deal with these soon

In [ ]:
iris_mean_sum = iris_csv.groupby('species').agg(['mean', 'sum'])
iris_mean_sum

## Using different aggregations for different columns

Here - we are interested in having different types of aggregations for different columns. We will look at calculating:

* The mean of the `sepal_length` per species
* The sum of the `sepal_width` per species

In order to do this, we pass in dictionaries. However, in the first case, note that we are losing the vital information about the levels of aggretation that are being performed:

In [ ]:
iris_csv.groupby('species').agg({'sepal_length': 'mean', 'sepal_width': 'sum'})

#### This is a quirk of using Pandas

In order to get around this - we need to enter in the aggregations that we want using lists. This way, we end up with having the multi level columns again

In [ ]:
iris_csv.groupby('species').agg({'sepal_length': ['mean'], 'sepal_width': ['sum']})

#### Getting rid of the multi levels

Here is a neat trick that you can use if you do not wish to have the multi levels. We will also add in some standard deviation info as well:

In [ ]:
multi_agg_df = iris_csv.groupby('species').agg({'sepal_length': ['mean'], 'sepal_width': ['sum', 'std']})

In [ ]:
multi_agg_df.columns

In [ ]:
multi_agg_df.columns = ['_'.join(x) for x in multi_agg_df.columns.ravel()]

In [ ]:
multi_agg_df

#### Custom aggregations

We do not have to rely on using built in aggregations on our grouped data. We can also use _anonymous_ functions, which in Python are called using `lambda`

Here we calculate the mean of values greater than 1

In [ ]:
iris_csv.groupby('species').agg(lambda x: x[x > 1].mean())

#### But the column names!

Another Pandas quirk here - if we want to add in the information about our custom aggregation - we need to pass in a __list of tuples!!__

In [ ]:
iris_csv.groupby('species').agg([('mean > 1', lambda x: x[x > 1].mean())])

## Pivot Tables

Here is a quick example of a common piece of functionality that is often used in Excel - the Pivot Table.

This is another way of aggregating data - but note now that the dataframe is transposed from what was there before.

In [ ]:
iris_csv.pivot_table(columns='species', aggfunc='sum')

We can also use lists to keep the information in multi level columns - and perform multiple types of aggregation as well.

In [ ]:
iris_csv.pivot_table(columns='species', aggfunc=['mean'])

In [ ]:
iris_csv.pivot_table(columns='species', aggfunc=['mean', 'sum'])

In [ ]:
iris_pivot = iris_csv.pivot_table(columns='species', aggfunc=['mean', 'sum'])
iris_pivot

#### Another column hack!

Notice the reordering of the levels - the `[::-1]` slice means 'traverse from the end to the start'

Don't worry if you are not familiar with this!

In [ ]:
iris_pivot.columns = ['_'.join(x[::-1]) for x in iris_pivot.columns.ravel()]

In [ ]:
iris_pivot

In [ ]:
li = [1, 2, 3, 4]

In [ ]:
li

In [ ]:
li[:]

In [ ]:
li[::]

In [ ]:
li[::1]

In [ ]:
li[::-1]

# Multi Level GroupBys

Here, we will show that we can use the `groupby` on multiple levels. The iris dataset is not suitable for this as it only has the species column to perform any meaningfull operations on. So we will use the diamonds data that we loaded in from Excel earlier.

Here, I was to get the mean value of all the numerical columns based on the 2 columns:

* `cut`
* `clarity`

In [ ]:
diamonds.head()

Note that we now get a `MultiIndex`!

In [ ]:
diamonds_mean = diamonds.groupby(['cut', 'clarity']).mean()

In [ ]:
diamonds_mean

## Getting information from `MultiIndex`

This can be quite a difficult topic, and is a bit more advanced, but I wanted to show you here as a future reference.

1. Drop the index and then use `loc`

In [ ]:
diamonds_mean2 = diamonds_mean.reset_index()
diamonds_mean2.loc[diamonds_mean2['cut'] == 'Fair']

2. We can use `xs` to get a cross section. But note that this can only be used on single levels within the index

In [ ]:
diamonds_mean.xs('Fair', level='cut')

In [ ]:
diamonds_mean.xs('IF', level='clarity')

In [ ]:
diamonds_mean.xs(('Good', 'VS2'), level=('cut', 'clarity'))

3. For multiple levels in multiple columns - we need to use an `IndexSlice` - __this is complicated!__

In [ ]:
idx = pd.IndexSlice

In [ ]:
diamonds_mean.loc[idx[['Good', 'Fair'], ['IF', 'SI1']], ['depth', 'price']]

# Time Series data and Plotting from Pandas

As the origins of Pandas are in the financial sector, there is a large amount of functionality for time based data. In this type of data, the index  really starts to play a key role, and becuase a `DatetimeIndex`.

In order to illustrate this, we will be loading in some financial data that was downloaded from Yahoo Finance. It is the Open/High/Low/Close data from Tesla for the past year.

Note that we are now specifying the column to use as an index. However - this is not enough to ensure a `DatetimeIndex`

In [ ]:
tsla = pd.read_csv('data/TSLA.csv', index_col='Date')

In [ ]:
tsla.info()

## `parse_dates`

The following command instructs Pandas to read in data as dates - if this can be done.

In this situation it works, as the data is formatted so that it can be easily read in as a date. However, if this information was more bespoke, then you will have to set a value for the `date_parser` argument using additional information from the `datetime` built in Python library. However, this is outside of the scope of this workshop.

In [ ]:
tsla = pd.read_csv('data/TSLA.csv', index_col='Date', parse_dates=True)

In [ ]:
tsla.info()

## Plotting the information

When working in a notebook - be sure that you have set `%matplotlib inline` before you do any plotting.

After this is done - to plot a Pandas series, simply call the `plot` function.

In [ ]:
tsla['Adj Close'].plot()

The default size is a little small, so we can add in the figsize argument

In [ ]:
tsla['Adj Close'].plot(figsize=(16, 8))

If we plot a dataframe instead - we get legend information

In [ ]:
tsla[['Adj Close']].plot(figsize=(16, 8))

## Plotting multiple lines

This is as simple as passing in multiple column names. Note that Pandas required the data to be in 'wide' format to do this

In [ ]:
tsla[['Open', 'Close']].plot(figsize=(16, 8))

## Multiple assignments and transform

Here we want to get information about the starting price of the stock each month. In order to do this:

* We first need to get the year and month information from the index
* The we perform a `transform` operation based on the year/month groupings
    * This is called a 'Window Function' - and also makes use of group by
    * Here we are using a built in function called `first`
    * We are also explicitly sorting by the index to ensure the ordering is correct
    * This might not be necessary - but I like to be sure!

In [ ]:
tsla['year'] = tsla.index.year

In [ ]:
tsla['month'] = tsla.index.month

In [ ]:
tsla['ACMS'] = tsla.sort_index().groupby(['year', 'month'])['Adj Close'].transform('first')

In [ ]:
tsla[['Adj Close', 'ACMS']].plot(figsize=(16, 8))

## Multiple assignments

It is possible to do all of the new columns in a single chain of functions using the `assign` operator

In [ ]:
tsla = pd.read_csv('data/TSLA.csv', index_col='Date', parse_dates=True)

In [ ]:
tsla_first = (
    tsla
    .assign(year = lambda df: df.index.year)
    .assign(month = lambda df: df.index.month)
    .assign(ACMS = lambda df: df.groupby(['year', 'month'])['Adj Close'].transform('first'))
)

In [ ]:
tsla_first[['ACMS', 'Adj Close']].plot(figsize=(16, 8))

If you wish to use strings for the column names instead, you can use the `**` syntax to 'unpack' a dictionary into keyword arguments

In [ ]:
tsla_first = (
    tsla
    .assign(**{'year': lambda df: df.index.year, 'month': lambda df: df.index.month})
    .assign(**{'ACMS': lambda df: df.groupby(['year', 'month'])['Adj Close'].transform('first')})
)

In [ ]:
tsla_first[['ACMS', 'Adj Close']].plot(figsize=(16, 8))

Do some joins and melt/pivot